# Analisi 2

- Data preparation

In [1]:
import pandas as pd
import numpy as np
import os

# Data Import

## Reviews

### Importo il dataset Reviews

In [41]:
folder_path = os.path.abspath("..")
data_dir = 'dataset/rome-airbnb'
reviews_filename = 'reviews.csv'
reviews_file = os.path.join(folder_path, data_dir, reviews_filename)

sample_dim = None # 100000

reviews_raw = pd.read_csv(reviews_file).head(sample_dim)
reviews_raw.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2737,24343517,2014-12-26,24333039,Raphaell,"This spot was in a great and nice area,walking..."
1,2737,24966779,2015-01-04,10493776,Ia,"Elif was really very helpful, while arranging ..."
2,2737,29770086,2015-04-12,29630601,Julia,"Elif's flat is spacious, clean, and had everyt..."
3,2737,30477570,2015-04-22,31215535,Sevil,I had a wonderful time staying with Elif and h...
4,2737,31659336,2015-05-08,1959813,Suzy,"Elif was a brilliant host, with an exceptional..."


In [43]:
reviews_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1113360 entries, 0 to 1113359
Data columns (total 6 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   listing_id     1113360 non-null  int64 
 1   id             1113360 non-null  int64 
 2   date           1113360 non-null  object
 3   reviewer_id    1113360 non-null  int64 
 4   reviewer_name  1113360 non-null  object
 5   comments       1112960 non-null  object
dtypes: int64(3), object(3)
memory usage: 51.0+ MB


In [44]:
reviews_raw.columns.to_list()

['listing_id', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments']

### Porto le date in formato datetime

In [45]:
reviews = reviews_raw.assign(date = pd.to_datetime(reviews_raw['date']))
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2737,24343517,2014-12-26,24333039,Raphaell,"This spot was in a great and nice area,walking..."
1,2737,24966779,2015-01-04,10493776,Ia,"Elif was really very helpful, while arranging ..."
2,2737,29770086,2015-04-12,29630601,Julia,"Elif's flat is spacious, clean, and had everyt..."
3,2737,30477570,2015-04-22,31215535,Sevil,I had a wonderful time staying with Elif and h...
4,2737,31659336,2015-05-08,1959813,Suzy,"Elif was a brilliant host, with an exceptional..."


## Calendar


### Importo il dataset Calendar

In [46]:
calendar_file = os.path.join(folder_path, data_dir, 'calendar.csv')

calendar_raw = pd.read_csv(calendar_file).head(sample_dim)
calendar = calendar_raw.assign(date = pd.to_datetime(calendar_raw['date']))
calendar.head()


,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,158668,2021-02-13,t,$40.00,$40.00,2.0,365.0
1,78314,2021-02-13,f,$69.00,$69.00,1.0,99.0
2,78314,2021-02-14,f,$69.00,$69.00,1.0,99.0
3,78314,2021-02-15,f,$69.00,$69.00,1.0,99.0
4,78314,2021-02-16,f,$69.00,$69.00,1.0,99.0


In [51]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10092245 entries, 0 to 10092244
Data columns (total 7 columns):
 #   Column          Dtype         
---  ------          -----         
 0   listing_id      int64         
 1   date            datetime64[ns]
 2   available       object        
 3   price           object        
 4   adjusted_price  object        
 5   minimum_nights  float64       
 6   maximum_nights  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 539.0+ MB


# Data prepation

## Creo le variabli di aggregazione mese e anno

In [52]:
%time
reviews['year'] = reviews['date'].dt.year
reviews['month'] = reviews['date'].dt.month
reviews = reviews.sort_values(['year', 'month'], ascending=False)
reviews.head()

CPU times: user 14 µs, sys: 1e+03 ns, total: 15 µs
Wall time: 272 µs


,listing_id,id,date,reviewer_id,reviewer_name,comments,year,month
4310,50471,729150541,2021-02-06,8582830,Anders,"Lovely place, super quiet and great atmosphere :)",2021,2
38546,335009,728807735,2021-02-04,95567665,Soraja,It was ideal. The Host is very responsive and ...,2021,2
66801,639627,728358403,2021-02-01,64638603,Nathalie,Massimo is kind and prompt in responding. The ...,2021,2
73665,721874,728694435,2021-02-03,210113502,Lubna,I liked my stay in Sabrina’s place. It was a w...,2021,2
88513,831199,728361293,2021-02-01,358777714,Carla,Che dire il sig. Pietro e la moglie persone mo...,2021,2


In [ ]:
%time
calendar['year'] = pd.DatetimeIndex(calendar['date']).year
calendar['month'] = pd.DatetimeIndex(calendar['date']).month
calendar = calendar.sort_values(['year', 'month'], ascending=False)
calendar.head()

## Preparare il prezzo come numero

Il prezzo è una stringa che contiene il sibolo della valuta `$` e una virgola come separatore delle migliaia. Bisogna quindi:

- Rimuovere il simbolo del dollaro
- Rimuovere il separatore delle migliaia
- Interpretare le stringhe come numeri reali


In [53]:
# PREVIOUS WAY
# @np.vectorize
# def remove_dollar(label: str):
#   return float(label.replace('$','').replace(',',''))

calendar_2 = calendar.assign(
        price          = pd.to_numeric(calendar.price.str.replace('$','').str.replace(',','')),
        # adjusted_price = pd.to_numeric(calendar.adjusted_price.str.replace('$','').str.replace(',','')),
    )
calendar_2['price'].head()

In [ ]:
calendar_2.head()

## Valori logici

In questo caso i valori logici si presentano come due stringhe 't' e 'f'. Noi preferiamo utilizzare le variabili booleane `True` e `False` che sono meglio comprese dal sitema, questo garantisce una serie di funzionalità in più: come l'utilzzo dell'algebra di Bool e il loro utilizzo come Masking.


In [ ]:
calendar_2['available'] = calendar_2.available.map({
    't': True,
    'f': False
})

## Valori derivati

In [ ]:
# Transformations (the mutable way)
calendar_2['available_prob'] = calendar_2.available.map({
    True: 1,
    False: 0
})

calendar_2['engaged_prob'] = 1 - calendar_2['available_prob']
calendar_2.head()


In [ ]:
# Transformations (the immutable way)
calendar_2 = calendar_2\
    .assign(
        available_prob=lambda _: _.available.map({
            True: 1,
            False: 0
        }),
        engaged_prob=lambda _: 1 - _.available_prob,
    )

calendar_2


# Analisi delle Attività

Gli ospiti che usano Airbnb possono lasciare una recensione dopo il loro pernottamento. Le recensioni possono essere usate come indicatori dell'attività e del tasso di attività.

La durata minima di pernottamento, il prezzo e il numero di recensioni sono stati usati come indicatori per stimare il tasso di occupazione, il numero di notti per anno e il rendita mensile di ogni annuncio.

Qual è il rapporto tra la rendita realizzata attraverso Airbnb e i prezzi degli affitti residenziali a lungo termine?

Il numero delle notti prenotate impedisce l'uso residenziale degli alloggi messi in affitto?

Qual è il rapporto tra la rendita realizzata attraverso AIRBNB e i prezzi degli affitti residenziali a lungo termine?




## Tasso di occupazione

### Creo le variabili che mi servono


### Conteggio Recensioni

In [ ]:
# Total reviews:
reviews.id.count()

### Recensioni / annunci / mese

In [ ]:

review_count = reviews\
  .groupby(['year', 'month', 'listing_id'], dropna=False)\
  .id.count()\
  .sort_index(ascending=False)\
  .reset_index()\
  .rename(columns={'id': 'count'})

review_count.head()


In [ ]:
# review_count['count_perc'] = (review_count['count'] / review_count['count'].sum() * 100).round(2)
# review_count['nights_per_year'] = (review_count['count'] / 365).round(2)

review_count['count'].mean()

### Fatturato al mese stimato (probabilmente sbagliato)

In [ ]:
# mean_day_price_engaged = calendar_2\
#     .assign(day_income = calendar_2.price * calendar_2.engaged_prob )\
#     .day_income\
#     .mean()
# mean_day_price_engaged * 30
    


In [ ]:
calendar

In [ ]:
reviews

In [ ]:
cal_1 = calendar.drop_duplicates(['listing_id', 'price', 'year', 'month'], keep='first')
cal_1

In [ ]:
reviews.merge(cal_1, on=['listing_id', 'year', 'month'])

# Proporzione di disponibilità

# Andamento del prezzo

## Periodo dell'anno

In [49]:
cal_1 = calendar_2[['listing_id', 'date', 'available', 'price', 'year', 'month']]
cal_1.head(10000)

,listing_id,date,available,price,year,month
398,2737,2022-02-01,True,18.0,2022,2
399,2737,2022-02-02,True,18.0,2022,2
400,2737,2022-02-03,True,18.0,2022,2
401,2737,2022-02-04,True,30.0,2022,2
402,2737,2022-02-05,True,30.0,2022,2
...,...,...,...,...,...,...
321583,860090,2022-02-03,True,99.0,2022,2
321584,860090,2022-02-04,True,99.0,2022,2
321585,860090,2022-02-05,True,99.0,2022,2
321586,860090,2022-02-06,True,99.0,2022,2


In [50]:
mean_price = cal_1\
    .groupby(['listing_id', 'month'])\
    .agg(
        mean_price = ( 'price', lambda x: round(np.mean(x), 2) ),
        std_price  = ( 'price', lambda x: round(np.std(x, ddof=1), 0) )
    )\
    .reset_index()

mean_price.sort_values(['std_price', 'mean_price'], ascending=False).head(10000)

,listing_id,month,mean_price,std_price
101977,12510273,2,4320.00,5009.0
102049,12528872,2,4322.86,5007.0
230905,33103707,2,3986.61,4955.0
235969,33772726,2,3976.61,4955.0
235981,33772935,2,3966.61,4955.0
...,...,...,...,...
217861,31132032,2,114.96,27.0
99434,12172089,3,114.52,27.0
308092,44027337,5,112.90,27.0
126001,16384532,2,112.89,27.0


In [ ]:
mean_price\
    .groupby(['listing_id'])\
    .agg(
        mean_price = ('mean_price', lambda x: round(np.mean(x), 0)),
        std_price  = ('mean_price', lambda x: round(np.std(x, ddof=1), 1) )
    )\
    .reset_index()\
    .sort_values(['std_price', 'mean_price'], ascending=False)

In [ ]:
# Get the price changes of a listing
cal_1[cal_1.listing_id == 20587]\
    .sort_values(['listing_id', 'year', 'month'])\
    .drop_duplicates(['listing_id', 'price'])

## Prezzo alloggi


In [11]:
folder_path = os.path.abspath("..")
data_dir = 'dataset/rome-airbnb'
listings_filename = 'listings.csv'
listings_filepath = os.path.join(folder_path, data_dir, listings_filename)

listings_raw = pd.read_csv(listings_filepath)
# Selezioniamo solo alcune delle colonne
listings_raw = listings_raw[[
    'id','name','longitude','latitude',
    'listing_url',
    'instant_bookable',
    'host_response_time',
    'host_neighbourhood',
    'review_scores_rating',
    'property_type',
    'room_type','accommodates',
    'bathrooms','bedrooms','beds','reviews_per_month','amenities',
    'number_of_reviews',
    'price'
  ]]

listings = listings_raw.assign(
        price          = lambda _: pd.to_numeric(_.price.str.replace('$','').str.replace(',',''))
    )
listings.head()

,id,name,longitude,latitude,listing_url,instant_bookable,host_response_time,host_neighbourhood,review_scores_rating,property_type,room_type,accommodates,bathrooms,bedrooms,beds,reviews_per_month,amenities,number_of_reviews,price
0,2737,"Elif's room in cozy, clean flat.",12.48355,41.87218,https://www.airbnb.com/rooms/2737,f,within a day,Testaccio,96.0,Private room,Private room,1,NaN,1.0,0.0,0.09,"[""Free parking on premises"", ""Elevator"", ""Drye...",7,21.0
1,2903,Stay a night in Calisto6 B&B Rome,12.47002,41.88826,https://www.airbnb.com/rooms/2903,f,within a few hours,Trastevere,86.0,Private room in bed and breakfast,Private room,4,NaN,1.0,2.0,0.37,"[""Shampoo"", ""Indoor fireplace"", ""TV"", ""Dedicat...",55,37.0
2,3079,Cozy apartment (2-4)with Colisseum view,12.49163,41.89503,https://www.airbnb.com/rooms/3079,f,NaN,Monti,88.0,Entire apartment,Entire home/apt,4,NaN,1.0,4.0,0.14,"[""Paid parking off premises"", ""Washer"", ""Dedic...",17,105.0
3,11834,"Roma ""Charming Boschetto House""",12.49058,41.89574,https://www.airbnb.com/rooms/11834,f,NaN,Monti,95.0,Entire apartment,Entire home/apt,2,NaN,1.0,1.0,1.19,"[""Pack \u2019n Play/travel crib"", ""Washer"", ""H...",142,90.0
4,12398,Casa Donatello - Home far from Home,12.46809,41.92584,https://www.airbnb.com/rooms/12398,f,within an hour,Flaminio,98.0,Entire apartment,Entire home/apt,5,NaN,2.0,3.0,0.32,"[""Dishwasher"", ""Lockbox"", ""Washer"", ""Dedicated...",40,73.0


In [12]:
pivot_df = listings\
    .groupby(['property_type','room_type'])\
    .price\
    .mean()\
    .round(0)\
    .reset_index()

pivot_df

,property_type,room_type,price
0,Boat,Entire home/apt,52.0
1,Camper/RV,Entire home/apt,52.0
2,Campsite,Entire home/apt,60.0
3,Casa particular,Entire home/apt,44.0
4,Castle,Entire home/apt,60.0
...,...,...,...
77,Shared room in townhouse,Shared room,12.0
78,Tent,Entire home/apt,28.0
79,Tiny house,Entire home/apt,68.0
80,Tipi,Entire home/apt,195.0


In [13]:
pivot_df\
    .pivot(
        columns='room_type',
        values='price',
        index=['property_type']
    )

room_type,Entire home/apt,Hotel room,Private room,Shared room
property_type,,,,
Boat,52.0,NaN,NaN,NaN
Camper/RV,52.0,NaN,NaN,NaN
Campsite,60.0,NaN,NaN,NaN
Casa particular,44.0,NaN,NaN,NaN
Castle,60.0,NaN,NaN,NaN
...,...,...,...,...
Shared room in townhouse,NaN,NaN,NaN,12.0
Tent,28.0,NaN,NaN,NaN
Tiny house,68.0,NaN,NaN,NaN


In [14]:
listings\
    .pivot_table(
        columns='room_type',
        values='price',
        index='property_type',
        aggfunc=lambda x: round(np.mean(x),0)
    )

room_type,Entire home/apt,Hotel room,Private room,Shared room
property_type,,,,
Boat,52.0,NaN,NaN,NaN
Camper/RV,52.0,NaN,NaN,NaN
Campsite,60.0,NaN,NaN,NaN
Casa particular,44.0,NaN,NaN,NaN
Castle,60.0,NaN,NaN,NaN
...,...,...,...,...
Shared room in townhouse,NaN,NaN,NaN,12.0
Tent,28.0,NaN,NaN,NaN
Tiny house,68.0,NaN,NaN,NaN


In [15]:
listings\
    .pivot_table(
        columns='bedrooms',
        values='price',
        index='host_neighbourhood',
        aggfunc=lambda x: round(np.mean(x),0)
    )

bedrooms,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,14.0,16.0
host_neighbourhood,,,,,,,,,,,,,,
Affori,NaN,70.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Astoria,90.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aurelio,81.0,118.0,191.0,226.0,260.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Balduina,69.0,145.0,112.0,127.0,150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Barona,NaN,280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Westpunt,112.0,216.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zona 3,59.0,116.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zona 7,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
